# 🕒 Ejercicios InfluxDB

Este cuaderno está diseñado para explorar **el manejo del tiempo y las ventanas** utilizando SQL sobre InfluxDB. Aprenderás a trabajar con **series temporales**, ventanas fijas y móviles, y cómo agregar datos en diferentes intervalos.

> **Requisitos previos**:
> - Tener una instancia de **InfluxDB 3** (Cloud Serverless/Dedicated, Core o Enterprise).  
> - Contar con un **token** con permisos de **lectura y escritura**.  

## Pasos previos

In [ ]:
# Instalar dependencias (si es necesario)
!pip install influxdb3-python pandas matplotlib

In [ ]:
# Montamos la carpeta (nos pedirá permisos)
from google.colab import drive
drive.mount('/content/drive')
# Crea un atajo llamado 'workspace' en la carpeta /content (dará un pequeño error si ya existe)
!ln -s "/content/drive/MyDrive/Colab Notebooks" "/content/workspace" >/dev/null 2>&1
# Ya podemos acceder a los ficheros:
data_path = "/content/workspace/data/"

## 1) Conexión: `InfluxDBClient3`

In [13]:
# Configuración de la conexión a InfluxDB

# URL de la instancia de InfluxDB
HOST = "URL_DE_INFLUXDB"
# Organización (solo necesario en InfluxDB Cloud)
ORG = "NOMBRE_DE_LA_ORG"
# Base de datos (en InfluxDB Cloud se denomina bucket y es necesario crearlo antes)
DB = "NOMBRE_DEL_BUCKET/BASE_DE_DATOS"
# Contraseña de acceso
TOKEN = "TOKEN_DE_INFLUXDB"

In [3]:
from influxdb_client_3 import InfluxDBClient3

client = InfluxDBClient3(host=HOST, token=TOKEN, org=ORG, database=DB)

## 2) Ejercicios

### Carga de datos y visualización

### `football_events`

| Columna  | Tipo       | Descripción                                                 |
|----------|------------|-------------------------------------------------------------|
| time     | TIMESTAMP  | Instante del evento (UTC)                                   |
| league   | STRING     | “LaLiga”, “Premier League”, “Serie A”, etc.                 |
| season   | STRING     | “2025/26”                                                   |
| match_id | STRING     | Identificador de partido (simulado)                         |
| team     | STRING     | Equipo real (p. ej., “Real Madrid”)                         |
| opponent | STRING     | Rival                                                       |
| player   | STRING     | Jugador real (p. ej., “Jude Bellingham”)                    |
| event    | STRING     | Tipo de evento: shot, goal, pass, foul, save, xg_event, etc.|
| value    | DOUBLE     | Métrica numérica (pases = 1, xG entre 0–1, velocidad, etc.) |
| minute   | INT        | Minuto simulado (0–90)                                      |
| venue    | STRING     | “home” / “away”                                             |

In [8]:
client.write_file(file=data_path + "football_events.csv", tag_columns=["tournament", "team", "opponent", "scorer"],data_format="csv")

query = """
SHOW COLUMNS IN football_events;
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,table_catalog,table_schema,table_name,column_name,data_type,is_nullable
0,public,iox,football_events,event,Utf8,YES
1,public,iox,football_events,league,Utf8,YES
2,public,iox,football_events,match_id,Utf8,YES
3,public,iox,football_events,minute,Int64,YES
4,public,iox,football_events,opponent,"Dictionary(Int32, Utf8)",YES
5,public,iox,football_events,player,Utf8,YES
6,public,iox,football_events,season,Utf8,YES
7,public,iox,football_events,team,"Dictionary(Int32, Utf8)",YES
8,public,iox,football_events,time,"Timestamp(Nanosecond, None)",NO
9,public,iox,football_events,value,Float64,YES


### Ejercicio 1

Usando la serie `football_events`, escribe una consulta que:

1. Cuente el número de goles (`goal`) por **equipo**.  
2. Solo muestre los equipos que hayan marcado **2 goles o más**.
3. Agrupe los resultados por `team`.
4. Ordene los resultados de mayor a menor cantidad de goles.
5. Solo muestre los **5 primeros** resultados.

**Pista:**: Recuerda que `WHERE` filtra filas antes de la agregación, mientras que `HAVING` filtra después de agrupar.

#### 🔹 La cláusula `HAVING` en SQL

La cláusula `HAVING` se utiliza para **filtrar resultados después de una agregación**. Esto es diferente de `WHERE`, que filtra **filas individuales antes de agrupar**.

##### 🔑 Diferencias entre `WHERE` y `HAVING`

| Cláusula | Cuándo se aplica | Ejemplo |
|----------|----------------|---------|
| `WHERE`  | Antes de la agregación, filtra filas | `WHERE event = 'goal'` |
| `HAVING` | Después de la agregación, filtra grupos | `HAVING COUNT(*) >= 2` |

In [9]:
query = """
SELECT
  team,
  COUNT(*) AS goals
FROM football_events
WHERE event = 'goal'
GROUP BY team
HAVING goals >= 2
ORDER BY goals DESC
LIMIT 5;
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,team,goals
0,VfB Stuttgart,51
1,Manchester City,44
2,AS Monaco,42
3,Sevilla,39
4,AC Milan,39


### Ejercicio 2

Usando la serie `football_events`, escribe una consulta que:

1. Seleccione las columnas: `time`, `team`, `player`, `event` y `value`.
2. Filtre los registros para los **últimos 30 días**.
3. Solo incluya eventos de la liga **"LaLiga"**.
4. Solo considere eventos del tipo **"shot"**.
5. Ordene los resultados por `time` de forma ascendente.
6. Solo muestre los **10 primeros** resultados.

🔍 **Pista:** utiliza `NOW()` y `INTERVAL` para calcular los últimos 30 días.

#### 💡 ¿Qué son las funciones `NOW()` e `INTERVAL` en SQL?

En SQL, la función `NOW()` devuelve la **fecha y hora actuales** del sistema en formato `TIMESTAMP`, lo que nos permite trabajar con el tiempo en tiempo real dentro de nuestras consultas.  

Para manipular fechas y horas, usamos la palabra clave `INTERVAL`, que permite **sumar o restar unidades de tiempo específicas**, como días, horas o minutos.  

**Ejemplos prácticos:**

```sql
-- Fecha y hora exacta hace 7 días
SELECT NOW() - INTERVAL '7 days';

-- Fecha y hora dentro de 2 horas
SELECT NOW() + INTERVAL '2 hours';
```

In [10]:
query = """
SELECT time, team, player, event, value
FROM football_events
WHERE time >= now() - INTERVAL '30 days'
  AND league = 'LaLiga'
  AND event = 'shot'
ORDER BY time ASC
LIMIT 10;
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,time,team,player,event,value
0,2025-10-29 16:47:19,Real Madrid,Vinicius Junior,shot,0.252
1,2025-10-29 16:47:27,Atletico de Madrid,Antoine Griezmann,shot,0.133
2,2025-10-29 16:47:58,Atletico de Madrid,Memphis Depay,shot,0.070
3,2025-10-29 16:49:18,Atletico de Madrid,Memphis Depay,shot,0.139
4,2025-10-29 16:51:26,Atletico de Madrid,Memphis Depay,shot,0.095
5,2025-10-29 16:52:50,Atletico de Madrid,Memphis Depay,shot,0.049
6,2025-10-29 16:55:12,Real Sociedad,Martin Zubimendi,shot,0.104
7,2025-10-29 16:56:23,Real Sociedad,Martin Zubimendi,shot,0.148
8,2025-10-29 16:57:27,Real Madrid,Vinicius Junior,shot,0.218
9,2025-10-29 16:57:40,Real Sociedad,Brais Mendez,shot,0.181


### Ejercicio 3

Usando la serie `football_events`, escribe una consulta que:

1. Cuente la cantidad de goles por equipo en los **últimos 5 días**.
2. Solo considere eventos del tipo **"goal"**.
3. Agrupe los resultados por `team`.
4. Ordene los equipos de mayor a menor cantidad de goles.
5. Solo muestre los **5 primeros** resultados.

🔍 **Pista:** utiliza `NOW()` y `INTERVAL` para filtrar los últimos 5 días, y `COUNT(*)` para contar los goles.

In [11]:
query = """
SELECT team, COUNT(*) AS goals
FROM football_events
WHERE time >= NOW() - INTERVAL '5 days'
  AND event = 'goal'
GROUP BY team
ORDER BY goals DESC
LIMIT 5;
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,team,goals
0,AS Monaco,25
1,Olympique Marseille,23
2,Lyon,21
3,RB Leipzig,20
4,Bayern Munich,15


### Ejercicio 4

Usando la serie `football_events`, escribe una consulta que:

1. Agrupe los eventos en ventanas de **15 minutos** usando `DATE_BIN`.  
2. Considere solo los eventos del **último día**.  
3. Calcule el **número de tiros (shots)** por equipo en cada ventana.  
4. Agrupe los resultados por `team` y por ventana.  
5. Ordene los resultados por `team` y por ventana.
6. Solo muestre los **10 primeros** resultados.

🔍 **Pista:** debes usar ventanas fijas.

#### 🕒 Ventanas Fijas (Tumbling Windows)

En procesamiento de **streaming** o series temporales, a menudo necesitamos **agrupar eventos en intervalos de tiempo** para analizarlos. Las **ventanas fijas**, también llamadas **tumbling windows**, son intervalos de tiempo **no solapados** y de duración constante. Cada evento pertenece **exactamente a una ventana**, y al terminar la ventana, se realiza el cálculo agregado (suma, promedio, conteo, etc).  

##### 🔑 Características principales:
- ⏱ **Duración fija:** por ejemplo, 5 minutos, 1 hora, 1 día.  
- 🚫 **No se superponen:** cada evento entra en **una sola ventana**.  
- 📊 **Útiles para métricas periódicas** y análisis histórico por bloques de tiempo.  

##### 📌 Ejemplo conceptual:
- Ventana de 1 hora:  
  - 🟢 Eventos de 10:00:00 a 10:59:59 → ventana **10:00–11:00**  
  - 🟢 Eventos de 11:00:00 a 11:59:59 → ventana **11:00–12:00**  

💡 **Tip:** Son ideales cuando quieres reportes o agregaciones **cada intervalo fijo**, como goles por hora, visitas por día o transacciones por minuto.

##### 💻 Ejemplo SQL de Ventanas Fijas

Supongamos que queremos **contar los goles por equipo cada 30 minutos**:

```sql
SELECT
    DATE_BIN(INTERVAL '30 minutes', time, TIMESTAMP '2025-01-01T00:00:00Z') AS window_start,
    team,
    COUNT(*) AS goals
FROM football_events
WHERE event = 'goal'
GROUP BY window_start, team
ORDER BY window_start, goals DESC;
```

🔍 Explicación del código:

- `DATE_BIN(INTERVAL '30 minutes', time, TIMESTAMP '2025-01-01T00:00:00Z')`: Agrupa cada evento en ventanas de 30 minutos, empezando desde la fecha base `2025-01-01T00:00:00Z`.
- `WHERE event = 'goal'`: Filtra por evento de tipo goal.
- `GROUP BY team, DATE_BIN(...)`: Agrupa por equipo y ventana.
- `COUNT(*) AS goals`: Cuenta cuántos goles hubo en esa ventana.
- `ORDER BY window_start, goals DESC`: Ordena los resultados por ventana y cantidad de goles.

💡 Este patrón es muy útil para análisis de series temporales en Big Data.

In [ ]:
query = """
SELECT
  DATE_BIN(INTERVAL '15 minutes', time, TIMESTAMP '2025-01-01T00:00:00Z') AS window_start,
  team,
  COUNT(*) AS shots
FROM football_events
WHERE event = 'shot'
    AND time >= now() - INTERVAL '1 days'
GROUP BY team, window_start
ORDER BY team, window_start
LIMIT 10;
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,window_start,team,shots
0,2025-11-13 16:45:00,Bayern Munich,5
1,2025-11-13 17:00:00,Bayern Munich,7
2,2025-11-13 17:15:00,Bayern Munich,4
3,2025-11-13 17:30:00,Bayern Munich,4
4,2025-11-13 17:45:00,Bayern Munich,6
5,2025-11-13 18:00:00,Bayern Munich,2
6,2025-11-13 17:00:00,Borussia Dortmund,8
7,2025-11-13 17:15:00,Borussia Dortmund,6
8,2025-11-13 17:30:00,Borussia Dortmund,5
9,2025-11-13 17:45:00,Borussia Dortmund,4


### Ejercicio 5

Usando la serie `football_events`, escribe una consulta que:

1. Agrupe los eventos de tipo **"shot"** en ventanas de **30 minutos** usando `DATE_BIN`.  
2. Considere solo los eventos de los **últimos 5 días**.  
3. Para cada equipo y ventana, calcule:  
   - `total_shots`: el **número total de tiros**.  
   - `last_shot_time`: la **hora del último tiro** dentro de la ventana.  
4. Agrupe los resultados por `team` y por ventana.  
5. Ordene los resultados por `team` y por ventana.
6. Solo muestre los **10 primeros** resultados.  

🔍 **Pista:**  
- Usa `COUNT(*)` para contar los tiros.  
- Puedes usar funciones como `selector_last(time, time)['time']` para obtener el último timestamp en la ventana.  


#### 🔹 Funciones `selector_*` en SQL

Las funciones `selector_*` son un conjunto de funciones muy útiles en análisis de **series temporales** o **ventanas**, que permiten **seleccionar un valor específico dentro de un grupo** basándose en un criterio de orden. Son ideales cuando no solo quieres agregados simples como `SUM` o `COUNT`, sino también **valores extremos o específicos** dentro de la ventana.

##### 🔑 Principales funciones `selector_*`:

| Función                            | Descripción                                                                 |
|------------------------------------|-----------------------------------------------------------------------------|
| `selector_first(valor, orden_columna)` | Devuelve el **primer valor** de `valor` según el orden definido en `orden_columna`. |
| `selector_last(valor, orden_columna)`  | Devuelve el **último valor** de `valor` según el orden definido en `orden_columna`. |
| `selector_min(valor, orden_columna)`   | Devuelve el valor **mínimo** de `valor` según el orden definido en `orden_columna`. |
| `selector_max(valor, orden_columna)`   | Devuelve el valor **máximo** de `valor` según el orden definido en `orden_columna`. |

##### 💻 Ejemplos prácticos:

1. **Último tiro por equipo en una ventana de 30 minutos:**

    ```sql
    selector_last(time, time)['time'] AS last_shot_time
    ```

2. **Primer jugador que hizo un pase en una ventana:**

    ```sql
    selector_first(player, time)['player'] AS first_passer
    ```

##### 🔍 ¿Cómo funcionan?

- Se define un **grupo o ventana**, por ejemplo, con `GROUP BY team, window_start`.
- La función recorre todos los registros dentro del grupo.
- Según la columna de orden (`orden_columna`), selecciona el valor específico deseado.
- Devuelve un registro estructurado, de donde normalmente se extrae el valor usando `['columna']`.


💡 **Tip**: Perfectas para análisis de **streaming** o **Big Data**, donde necesitas obtener el primer, último, mínimo o máximo valor dentro de cada ventana de tiempo, además de los agregados tradicionales como `SUM`, `COUNT`, `AVG`.


In [70]:
query = """
SELECT
  DATE_BIN(INTERVAL '30 minutes', time, TIMESTAMP '1970-01-01T00:00:00Z') AS window_start,
  team,
  COUNT(*) AS total_shots,
  selector_last(time, time)['time'] AS last_shot_time
FROM football_events
WHERE event = 'shot'
  AND time >= now() - INTERVAL '1 day'
GROUP BY window_start, team
ORDER BY team, window_start
LIMIT 10;
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,window_start,team,total_shots,last_shot_time
0,2025-11-13 16:30:00,Bayern Munich,5,2025-11-13 16:58:24
1,2025-11-13 17:00:00,Bayern Munich,11,2025-11-13 17:26:44
2,2025-11-13 17:30:00,Bayern Munich,10,2025-11-13 17:57:02
3,2025-11-13 18:00:00,Bayern Munich,2,2025-11-13 18:12:45
4,2025-11-13 17:00:00,Borussia Dortmund,14,2025-11-13 17:27:18
5,2025-11-13 17:30:00,Borussia Dortmund,9,2025-11-13 17:56:57
6,2025-11-13 18:00:00,Borussia Dortmund,3,2025-11-13 18:10:13
7,2025-11-13 16:30:00,FC Barcelona,1,2025-11-13 16:58:58
8,2025-11-13 17:00:00,FC Barcelona,7,2025-11-13 17:28:28
9,2025-11-13 17:30:00,FC Barcelona,5,2025-11-13 17:54:16


#### Ejercicio 6

Usando la serie `football_events` y Python, escribe una consulta que:

1. Seleccione las columnas `time` y `player`.  
2. Calcule el **número de pases de cada jugador en los últimos 20 minutos** usando una ventana móvil (`COUNT(*) OVER (...)`).  
3. Filtre solo los eventos de tipo **"pass"** en los últimos 7 días.  
4. Ordene los resultados por jugador y tiempo.  
5. Devuelva únicamente las primeras 10 filas.

🔍 **Pista:**  
- Usa `PARTITION BY player` para calcular la ventana individual por jugador.  
- Usa `RANGE INTERVAL '20 minutes' PRECEDING` para definir la ventana móvil de 20 minutos.

#### 🏃‍♂️ Ventanas Móviles (Sliding Windows)

En análisis de **streaming** o series temporales, a menudo es necesario analizar **eventos recientes de manera continua**, en lugar de trabajar con bloques fijos de datos. Para esto, usamos las **ventanas móviles** (sliding windows), que:

- Tienen una **duración fija** (por ejemplo, 30 minutos).
- Se **mueven continuamente** a intervalos más pequeños (por ejemplo, cada minuto).
- Pueden **solaparse**, lo que significa que un mismo evento puede pertenecer a varias ventanas consecutivas.

##### 🔑 Diferencias con las ventanas fijas

| Característica                        | Ventanas Fijas               | Ventanas Móviles             |
|---------------------------------------|------------------------------|------------------------------|
| **Duración**                          | Fija                         | Fija                         |
| **Solapamiento**                      | No                           | Sí                           |
| **Cada evento pertenece a**          | Una sola ventana             | Varias ventanas posibles     |
| **Ejemplo de uso**                    | Conteo de goles por hora     | Promedio de tiros en los últimos 30 minutos, actualizado cada minuto |

##### 📌 Ejemplo conceptual

```sql
SELECT
  time,
  player,
  COUNT(*) OVER (
    PARTITION BY player
    ORDER BY time
    RANGE INTERVAL '30 minutes' PRECEDING
  ) AS goals_last_30m
FROM football_events
WHERE event = 'goal'
  AND time >= NOW() - INTERVAL '7 days'
ORDER BY player, time;
```

🔍 Explicación del código:

- `PARTITION BY player`: Agrupa los datos por jugador, para calcular la ventana móvil individualmente para cada jugador.
    - `ORDER BY time`: Ordena los goles por timestamp dentro de cada jugador, para que la ventana tenga sentido temporal.
    - `RANGE INTERVAL '30 minutes' PRECEDING`: Define la ventana móvil: incluye todos los goles de los últimos 30 minutos hasta el evento actual.
    - `COUNT(*) OVER (...) AS goals_last_30m`: Cuenta cuántos goles ha marcado el jugador en esa ventana de 30 minutos.
- `WHERE event = 'goal' AND time >= NOW() - INTERVAL '7 days'`: Filtra solo los eventos de goles y limita el análisis a los últimos 7 días.
- `ORDER BY player, time`: Ordena los resultados por jugador y tiempo para que se vea claramente la evolución.

💡 **Tip:** Las ventanas móviles son perfectas para **métricas en tiempo real**, como el promedio de tiros en los últimos 30 minutos, el conteo de eventos recientes o para alertas de actividad inusual.

In [72]:
query = """
SELECT
  time,
  player,
  COUNT(*) OVER (
    PARTITION BY player
    ORDER BY time
    RANGE INTERVAL '20 minutes' PRECEDING
  ) AS passes_last_20m
FROM football_events
WHERE event = 'pass'
  AND time >= now() - INTERVAL '7 days'
ORDER BY player, time
LIMIT 10;
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,time,player,passes_last_20m
0,2025-11-08 15:06:08,Achraf Hakimi,1
1,2025-11-08 15:07:20,Achraf Hakimi,2
2,2025-11-08 15:12:16,Achraf Hakimi,3
3,2025-11-08 15:14:16,Achraf Hakimi,4
4,2025-11-08 15:16:17,Achraf Hakimi,5
5,2025-11-08 15:24:55,Achraf Hakimi,6
6,2025-11-08 15:29:56,Achraf Hakimi,5
7,2025-11-08 15:33:19,Achraf Hakimi,5
8,2025-11-08 15:37:43,Achraf Hakimi,4
9,2025-11-08 15:40:50,Achraf Hakimi,5


## Cerrar conexión con InfluxDB

In [91]:
client.close()